<a href="https://colab.research.google.com/github/abhinao007/Code-tensorflow/blob/main/ML_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd 
from textblob import TextBlob
import matplotlib.pyplot as plt
from google.colab import files
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Activation,Embedding,Bidirectional

In [7]:


# Install Kaggle library
!pip install -q kaggle
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [9]:
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d alaix14/bitcoin-tweets-20160101-to-20190329


100% 1.58G/1.58G [00:14<00:00, 53.7MB/s]
100% 1.58G/1.58G [00:14<00:00, 117MB/s] 


In [11]:

from zipfile import ZipFile

file_name = "bitcoin-tweets-20160101-to-20190329.zip"

with ZipFile(file_name, "r") as zip:
  zip.extractall()
  print("done")

done


In [44]:
import pandas as pd
df = pd.read_csv('tweets.csv', sep=';',nrows=20000)

In [45]:
#predit the language to filter for English 
!pip install whatthelang
from whatthelang import WhatTheLang

wtl = WhatTheLang()
L=[]
for row in df['text']:
    if len(row)!=0:
        L.append(wtl.predict_lang(row))
    else:
        L.append(None)
        
df['lang'] = L
df.head()
df = df[df["lang"] == 'en']

In [3]:
#upload files for Bitcoin trading

uploaded = files.upload()

Saving BTC_USD_2013-10-01_2021-07-02-CoinDesk.csv to BTC_USD_2013-10-01_2021-07-02-CoinDesk.csv


In [15]:
df_btc = pd.read_csv("BTC_USD_2013-10-01_2021-07-02-CoinDesk.csv")
df_btc.head()

,Currency,Date,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
0,BTC,2013-10-01,123.65499,124.30466,124.75166,122.56349
1,BTC,2013-10-02,125.45500,123.65499,125.75850,123.63383
2,BTC,2013-10-03,108.58483,125.45500,125.66566,83.32833
3,BTC,2013-10-04,118.67466,108.58483,118.67500,107.05816
4,BTC,2013-10-05,121.33866,118.67466,121.93633,118.00566


In [18]:
df_btc["Date_Time"]=pd.to_datetime(df_btc["Date"], format='%Y%m%d', errors='ignore')

In [26]:
df_btc['Date_Time']= pd.to_datetime(df_btc['Date'])
df_btc.head()


,Currency,Date,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD),Date_Time
0,BTC,2013-10-01,123.65499,124.30466,124.75166,122.56349,2013-10-01
1,BTC,2013-10-02,125.45500,123.65499,125.75850,123.63383,2013-10-02
2,BTC,2013-10-03,108.58483,125.45500,125.66566,83.32833,2013-10-03
3,BTC,2013-10-04,118.67466,108.58483,118.67500,107.05816,2013-10-04
4,BTC,2013-10-05,121.33866,118.67466,121.93633,118.00566,2013-10-05


In [50]:
df['Date_Time']= pd.to_datetime(df['timestamp'], format='%Y-%m-%d')

df['Date_Time']= pd.to_datetime(df['Date_Time']).dt.date
df['Date_Time']= pd.to_datetime(df['Date_Time'])
df.head()

,id,user,fullname,url,timestamp,replies,likes,retweets,text,lang,Date_Time
1,1132977073402736640,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0,cardano digitize currencies eos co ktkqkebls r...,en,2019-05-27
2,1132977023893139456,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1,another test tweet caught stream bitcoin,en,2019-05-27
3,1132977089089556481,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0,current crypto prices btc usd eth usd ltc usd ...,en,2019-05-27
4,1132977092340191232,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0,spiv nosar baz bitcoin asset amp currency co f...,en,2019-05-27
6,1132977101638897665,evilrobotted,evilrobotted,NaN,2019-05-27 11:49:25+00,0,0,0,nwoodfine building real bitcoin sv building br...,en,2019-05-27


In [40]:
len(df)

11926

In [47]:
!pip install nltk
!pip install re

import nltk
import re
nltk.download("stopwords")
from nltk.corpus import stopwords

def text_cleaning(text):
    forbidden_words = set(stopwords.words('english'))
    text = ' '.join(text.split('.'))
    text = re.sub('\/',' ',text)
    text = text.strip('\'"')
    text = re.sub(r'@([^\s]+)',r'\1',text)
    text = re.sub(r'\\',' ',text)
    text = text.lower()
    text = re.sub('[\s]+', ' ', text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub(r'((http)\S+)','',text)
    text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z]', ' ', text.strip().lower())).strip()
    text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
    text = [word for word in text.split() if word not in forbidden_words]
    return ' '.join(text)

df['text'] = df['text'].apply(lambda text: text_cleaning(text))
df.sample(3)

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,user,fullname,url,timestamp,replies,likes,retweets,text,lang,Date_Time
5540,1126830222622564357,bitcoinmate,Bitcoin Mate (BTC News),NaN,2019-05-10 10:43:55+00,0,0,0,crypto wallet abra adds app support thousands ...,en,2019-05-10
4488,1126796716378918912,cryptoJunky6b,cryptoJunky,NaN,2019-05-10 10:28:38+00,0,5,1,alts good morning woke checked crypto first ki...,en,2019-05-10
14638,1126864479835295744,neoprice_,NEO Price,NaN,2019-05-10 13:00:02+00,0,0,0,neo worth btc neo co ifl bedsrv,en,2019-05-10


In [56]:
df_j = pd.merge(df, df_btc, how='left', on = 'Date_Time')

In [59]:
df_j.head()

,id,user,fullname,url,timestamp,replies,likes,retweets,text,lang,Date_Time,Currency,Date,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
0,1132977073402736640,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0,cardano digitize currencies eos co ktkqkebls r...,en,2019-05-27,BTC,2019-05-27,8800.346606,8653.151802,8937.246491,8621.572192
1,1132977023893139456,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1,another test tweet caught stream bitcoin,en,2019-05-27,BTC,2019-05-27,8800.346606,8653.151802,8937.246491,8621.572192
2,1132977089089556481,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0,current crypto prices btc usd eth usd ltc usd ...,en,2019-05-27,BTC,2019-05-27,8800.346606,8653.151802,8937.246491,8621.572192
3,1132977092340191232,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0,spiv nosar baz bitcoin asset amp currency co f...,en,2019-05-27,BTC,2019-05-27,8800.346606,8653.151802,8937.246491,8621.572192
4,1132977101638897665,evilrobotted,evilrobotted,NaN,2019-05-27 11:49:25+00,0,0,0,nwoodfine building real bitcoin sv building br...,en,2019-05-27,BTC,2019-05-27,8800.346606,8653.151802,8937.246491,8621.572192


In [60]:
from textblob import TextBlob

def sentiment(txt):
    return TextBlob(txt).sentiment.polarity

df_j['sentiment'] = df_j['text'].apply(lambda txt: sentiment(txt))      # new column of sentiment

df_j.sample(10)

,id,user,fullname,url,timestamp,replies,likes,retweets,text,lang,Date_Time,Currency,Date,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD),sentiment
3641,1126827139695824897,AlfathNafal,Crypto Signalist FRIEND,NaN,2019-05-10 10:31:40+00,0,0,0,rcn btc touched target achieved half day profi...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.116667
1510,1132982336960770048,jilatarduna,Leslie Fedorenko,NaN,2019-05-27 12:10:13+00,0,0,0,along amazing rewards users obx also plan brin...,en,2019-05-27,BTC,2019-05-27,8800.346606,8653.151802,8937.246491,8621.572192,0.300000
7651,1126847321407406081,TheCryptoPadawa,TheCryptoPadawan,NaN,2019-05-10 11:51:51+00,0,0,0,bitfinex users unable withdraw funds million d...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,-0.550000
4723,1126310803748028417,kerooke,Kevin Rooke,NaN,2019-05-09 00:19:56+00,13,217,57,bitcoin first hit usd oct bitcoin back changed...,en,2019-05-09,BTC,2019-05-09,6159.685902,5932.446285,6159.685902,5845.650918,0.125000
4460,1126830897418964992,CompumatrixUS,Compumatrix,NaN,2019-05-10 10:46:36+00,0,0,0,latest compumatrix today co vocxxp n thanks sa...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.350000
7638,1126845925496897539,SandraHappyOd,Sandra,NaN,2019-05-10 11:46:19+00,0,0,0,qkc blockchain btc eth blockchaintechnology,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.000000
11643,1126864476702167040,NapalmDeathBot,Napalm Death Bot,NaN,2019-05-10 13:00:02+00,0,0,0,suffer bitcoin hit time remotely toggle rig ho...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,-0.100000
12622,1126868469629313024,GayleU64,Gayle Ullmer,NaN,2019-05-10 13:15:54+00,0,0,0,dearmetenyearsago something called bitcoin lau...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.300000
8963,1127277847725264896,TheePossibility,Imri,NaN,2019-05-11 16:22:37+00,0,0,0,smh,en,2019-05-11,BTC,2019-05-11,7254.805536,6347.706737,7337.322181,6346.331889,0.000000
3638,1126827114915880961,tnwrss,TNWRSS,NaN,2019-05-10 10:31:34+00,0,0,0,francis pouliot separates bitcoin blockchain c...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.000000


In [61]:
from numpy.random import RandomState

rng = RandomState()
train_data = df_j.sample(frac=0.8, random_state=rng)
test_data = df_j.loc[~df_j.index.isin(train_data.index)]

La taille des données d entrinement: 12796
La taille des données de test: 3199


In [77]:
max_features = 20000  # Only consider the top 20k words
maxlen = 200

In [66]:
train_data['flag'] = 'TRAIN'
test_data['flag'] = 'TEST'


total_docs = pd.concat([train_data,test_data],axis = 0,ignore_index = True)
phrases = total_docs['text'].tolist()

total_docs.sample(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,user,fullname,url,timestamp,replies,likes,retweets,text,lang,Date_Time,Currency,Date,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD),sentiment,flag
5108,1126855830219845632,CharlieIsBig,Charlie Browner,NaN,2019-05-10 12:25:40+00,0,0,0,justinsuntron binance cz binance hero give btc...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.500000,TRAIN
10734,1126861571899252738,wanderingsadhak,the wandering sadhaka,NaN,2019-05-10 12:48:29+00,0,0,0,possible ban btc bitcoin congress rep sherman ...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,-0.050000,TRAIN
2328,1126830542073286656,BruceDayne,Khareem Sudlow,NaN,2019-05-10 10:45:11+00,0,0,0,cryptocurrency btcnews binance co ahyq kbo jpm...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.000000,TRAIN
5604,1116850088347746305,AfricanCoinExch,African Coin Exchange,NaN,2019-04-12 23:46:25+00,0,65,55,africancoinexchange cryptocurrency buybtc cryp...,en,2019-04-12,BTC,2019-04-12,5077.515864,5042.402341,5098.606039,4894.733180,0.000000,TRAIN
12779,1126860842702602240,deYummyEvents,deYummy Events,NaN,2019-05-10 12:45:35+00,0,0,0,latest deyummy sexsational events co ctciugibc...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.350000,TRAIN
15630,1126875252380897280,PrettyHatMech,🄿🅁🄴🅃🅃🅈 🄷🄰🅃 🄼🄰🄲🄷🄸🄽🄴,NaN,2019-05-10 13:42:51+00,0,0,0,khaleesi rises rationalwiki use paypal going s...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.000000,TEST
860,1126842347336105984,KevinEllah1,Kevin,NaN,2019-05-10 11:32:05+00,0,0,0,fidelity stick top cryptos like bitcoin xrp sa...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.500000,TRAIN
8646,1126859813617410049,SusantiNuris,nuris susanti,NaN,2019-05-10 12:41:30+00,0,0,0,win pxb amp btc sign awesome contest co pl jnb...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.658333,TRAIN
9646,1126869022744563712,MercadosBitcoin,Mercados ฿itcoin,NaN,2019-05-10 13:18:05+00,0,0,0,holding back bitcoin venezuela group investiga...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.000000,TRAIN
14634,1126853279009218564,PlanetaryDice,Demogorgon,NaN,2019-05-10 12:15:32+00,0,0,0,marvinchebbi thecryptomonk scrolling elf power...,en,2019-05-10,BTC,2019-05-10,6387.829450,6159.689544,6426.831207,6098.157976,0.400000,TEST


In [76]:
from keras.preprocessing.text import one_hot
vocab_size = 50000
encoded_phrases = [one_hot(d, vocab_size) for d in phrases]
total_docs['Phrase'] = encoded_phrases
train_data = total_docs[total_docs['flag'] == 'TRAIN']
test_data = total_docs[total_docs['flag'] == 'TEST']
x_train = train_data['Phrase']
y_train = train_data['sentiment']
x_val = test_data['Phrase']
y_val = test_data['sentiment']

In [78]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

In [79]:
model = Sequential()
inputs = keras.Input(shape=(None,), dtype="int32")

# Embed each integer in a 128-dimensional vector
model.add(inputs)
model.add(Embedding(50000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))

# Add a classifier
model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          1600000   
_________________________________________________________________
lstm (LSTM)                  (None, None, 32)          8320      
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 1,616,673
Trainable params: 1,616,673
Non-trainable params: 0
_________________________________________________________________


In [80]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=["accuracy"])

model.fit(x_train, y_train, 
          batch_size=128, 
          epochs=20, 
          validation_data=(x_val, y_val),
          validation_steps=20)

Epoch 1/20
100/100 [==============================] - 49s 280ms/step - loss: 0.3659 - accuracy: 0.4874 - val_loss: 0.2566 - val_accuracy: 0.4945
Epoch 2/20
100/100 [==============================] - 27s 269ms/step - loss: 0.1991 - accuracy: 0.4981 - val_loss: 0.1502 - val_accuracy: 0.4949
Epoch 3/20
100/100 [==============================] - 27s 265ms/step - loss: 0.0762 - accuracy: 0.4989 - val_loss: 0.0980 - val_accuracy: 0.4953
Epoch 4/20
100/100 [==============================] - 27s 267ms/step - loss: 0.0460 - accuracy: 0.4984 - val_loss: 0.6406 - val_accuracy: 0.4945
Epoch 5/20
100/100 [==============================] - 27s 267ms/step - loss: 0.2743 - accuracy: 0.4951 - val_loss: 0.0378 - val_accuracy: 0.4957
Epoch 6/20
100/100 [==============================] - 27s 268ms/step - loss: -0.0982 - accuracy: 0.4989 - val_loss: -0.0140 - val_accuracy: 0.4957
Epoch 7/20
100/100 [==============================] - 27s 267ms/step - loss: -0.1871 - accuracy: 0.5029 - val_loss: -0.0707 - va

In [81]:
test_loss, test_acc = model.evaluate(x=x_val, y=y_val)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

sample_text = ('Bitcoin just lost half its value overnight. Sorry all you savvy investors ')
vocab_size = 50000

model.predict(one_hot(sample_text, vocab_size))

100/100 [==============================] - 4s 36ms/step - loss: -0.6237 - accuracy: 0.4986
Test Loss: -0.623680591583252
Test Accuracy: 0.4985933005809784


array([[1.0762587e-01],
       [1.8603727e-01],
       [1.5957099e-01],
       [1.1550453e-09],
       [2.2997433e-01],
       [6.8857670e-02],
       [1.8781424e-03],
       [4.6504616e-11],
       [1.3946167e-01],
       [1.8456548e-02],
       [5.9124231e-03],
       [1.7454177e-02]], dtype=float32)

In [4]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [3]:
import nltk
import re
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
